##  Import the Libraries

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

## Read the Data

In [2]:
data = pd.read_csv("train.csv")
sc_X = StandardScaler()
Y = data["target"]
del data['target']
del data['id']
X = data
X =  pd.DataFrame(sc_X.fit_transform(X))
X = X.to_numpy()
Y = Y.to_numpy()
Y = np.array([Y])
Y = Y.reshape((250,1))


## Data Loader

In [14]:
class  Training_DataSet(Dataset):
    def __init__(self,csv_file,X,Y,transform):
        self.data = pd.read_csv(csv_file)
        self.X = X
        self.Y = Y
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        features = self.X[idx]  
        target = self.Y[idx]
        
        sample = {"features":features,"target":target}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    


class to_tensor_training(object):
    def __call__(self,sample):
        features , target = sample['features'],sample['target']
        features = np.array([features])
        features = features.astype('float32')
        features = torch.from_numpy(features)
        features = features.float()
        target = np.array([target])
        target = torch.from_numpy(target)
        target = target.long()
        
        return {"features":features,"target":target}
        

In [15]:
transformed_dataset = Training_DataSet(csv_file="train.csv",
                                       X = X,
                                       Y = Y,
                                       transform = to_tensor_training())

In [16]:
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['features'].sum(),sample['target'])

    if i == 3:
        break

0 tensor(-2.6132) tensor([[1]])
1 tensor(25.8242) tensor([[0]])
2 tensor(9.1408) tensor([[1]])
3 tensor(27.8698) tensor([[1]])


## Create a network

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(300,512)
        self.fc2 = nn.Linear(512,2)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [18]:
net = Net()

In [19]:
trainloader = torch.utils.data.DataLoader(transformed_dataset,batch_size=4, shuffle=True,num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:
next(iter(trainloader))

{'features': tensor([[[ 0.0569, -1.5429,  0.6263,  ..., -0.6655,  0.9250,  0.1248]],
 
         [[-0.4359, -0.7715,  1.2726,  ...,  0.4529,  1.0731,  0.8934]],
 
         [[-1.2268,  0.4893, -1.0547,  ...,  0.2754, -0.2628, -1.0390]],
 
         [[-0.3656, -0.2970,  0.0555,  ..., -0.6123, -0.0292, -0.3168]]]),
 'target': tensor([[[0]],
 
         [[0]],
 
         [[0]],
 
         [[1]]])}

## Training

In [21]:
loss_array = []
correct = 0
total = 0

accuracy_array = []
for epoch in range(11):
    running_loss = 0
    for i , data in enumerate(trainloader,0):
        features , target = data['features'],data['target']
        target = target.squeeze()
        optimizer.zero_grad()
        outputs = net(features)
        outputs = outputs.squeeze()
        _, predictions = torch.max(outputs,1)
        total += target.size(0)
        correct += (predictions == target).sum().item()
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
    loss_array.append(running_loss / len(trainloader))
    
    print("Epoch: {}/ 10.. ".format(epoch+1),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)))
    print("Accuracy  %d %%" %(100*correct/total))
    correct = 0
    total = 0
print("Finished training")

Epoch: 1/ 10..  Training Loss: 0.704.. 
Accuracy  58 %
Epoch: 2/ 10..  Training Loss: 0.533.. 
Accuracy  74 %
Epoch: 3/ 10..  Training Loss: 0.412.. 
Accuracy  88 %
Epoch: 4/ 10..  Training Loss: 0.318.. 
Accuracy  94 %
Epoch: 5/ 10..  Training Loss: 0.224.. 
Accuracy  99 %
Epoch: 6/ 10..  Training Loss: 0.157.. 
Accuracy  100 %
Epoch: 7/ 10..  Training Loss: 0.115.. 
Accuracy  100 %
Epoch: 8/ 10..  Training Loss: 0.087.. 
Accuracy  100 %
Epoch: 9/ 10..  Training Loss: 0.069.. 
Accuracy  100 %
Epoch: 10/ 10..  Training Loss: 0.055.. 
Accuracy  100 %
Epoch: 11/ 10..  Training Loss: 0.046.. 
Accuracy  100 %
Finished training


In [65]:
### TESTING